# Instructions

The following code was designed in order to implement batch processing of location tracking.  Currently, for this to work, all videos must take the same settings, including the defined regions of interest.  To confirm that regions of interest are consistent across videos, for each video an overlay of the reference frame, the regions of interest and the animal traace are produced.  Moreover, the reference frame is currently required to be generated by taking the median of each individual video.  In addition to saving frame by frame location, distance travelled, and whether the animal is in each ROI in separate csv files, bins can also be defined for summarizing videos (e.g. minute by minute).  All summary information will be saved in a single file.

### Package Requirements
Please see instructions under repository README for package requirements and install instructions.

# 1. Load Necessary Packages
The following code loads neccessary packages and need not be changed by the user.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import LocationTracking_Functions as lt
import holoviews as hv

# 2. User Defines Batch Processing Directory and Settings
Below, options are set by user for batch processing of videos.  If you are unfamiliar with these settings, please see LocationTracking_Individual.ipynb.
***Windows Users:*** Place an 'r' in front directory path (e.g. r"zp\Videos") to avoid mishandling of forward slashes.

In [ ]:
#define video paramaters
video_dict = {
    'dpath' : "/Users/zachpennington/Desktop/Videos/", # directory containing file
    'ftype' : "mpg", #file type. 
    'fps' : 30, #frames per second
    'start' : 0, #frame at which to start. 0-based
    'end' : None #frame at which to end.  set to None if processing whole video.
}

#define parameters for location tracking
tracking_params = {
    'loc_thresh' : 99,
    'use_window' : True, #True/False.  Will window surrounding prior location be imposed?
    'window_size' : 100, #The length of one side of square window.  
    'window_weight' : 1, #0-1 scale, where 1 is maximal weight of window surrounding prior locaiton.
}

#set bin_dict
#set bin_dict = None if only overall session average is desired
bin_dict = {
    1: (0, 30),
    2: (30, 60)
}

#set region_names
#set region_names to None if no ROIs are needed
region_names=['Left','Right']

#code below loads folder with files.  doesn't need to be changed
#crop and poly_stream are initialized as None. 
video_dict,crop,poly_stream = lt.Batch_LoadFiles(video_dict)
video_dict['FileNames']

# 3. (Optional) Crop Image if Desired
To crop video frame, after running code below, select box selection tool below image (square with a plus sign).  To start drawing region to be included in analyis, double click image.  Double click again to finalize region.  If you decide to change region, it is best to rerun this cell and subsequent steps.  Note that this is done based upon first video in folder.

In [ ]:
%%output size=100
#Select output size if image is too small/large.  Code above must be first line in cell and dictates overall size
#of image, where 100 is standard.  

image,crop,video_dict=lt.LoadAndCrop(video_dict,stretch,cropmethod='Box')
image

# 4. (Optional) Define Regions of Interest
**Do not run if region_names is set to None.**

After running cell below, draw regions of interest on presented image in the order you provided them.  To start drawing a region, double click on image.  Single click to add a vertex.  Double click to close polygon.  If you mess up it's easiest to re-run cell.  Note that this is done based upon first video in folder.

In [ ]:
%%output size=100
#Select output size if image is too small/large.  Code above must be first line in cell and dictates overall size
#of image, where 100 is standard.  

reference = lt.Reference(video_dict,crop,num_frames=100) #gets reference from first video
plot,poly_stream = lt.ROI_plot(reference,region_names,stretch) 
plot

# 5. Perform Batch Processing and Display Traces from Each Session

In [ ]:
%%opts Layout [shared_axes=False] 
%%output size=100
#Select output size if image is too small/large.  Code above must be first line in cell and dictates overall size
#of image, where 100 is standard.  

heatmaps = lt.Batch_Process(video_dict,tracking_params,bin_dict,region_names,stretch,crop,poly_stream)
heatmaps.cols(1)